<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2021notebooks/2021_0917RNN_demo_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- date: 2021_0917
- filename: 2021_0917RNN_demo_PyTorch.ipynb

In [1]:
# 言語モデル PyTorch 版
# original: github pytorch examples, word language model/main.py
import time
import math
import os
import torch
import torch.nn as nn
import torch.onnx

# Model parameters.
#parser.add_argument('--data', type=str, default='./data/wikitext-2',
#                    help='location of the data corpus')
#args_data='./data/wikitext-2'
args_data_basedir = '/Users/asakawa/study/2020STAIR-Lab-CIT_STAIR-captions.git/'
args_data_basedir = '.'
args_data = args_data_basedir
args_model='LSTM'            #RNN_TANH, RNN_RELU, LSTM, GRU, Transformer が選択可能
args_emsize=200              #単語埋め込み次元 size of word embeddings
args_nhid=200                #層ごとのニューロン数 number of hidden units per layer
args_nlayers=2               #層数 number of layers
args_lr=20                   #学習率の初期値 initial learning rate
args_clip=0.25               #勾配クリップの値 gradient clipping
args_epochs=40               #繰り返しの最大エポック数 upper epoch limit
args_batch_size=20           #ミニバッチサイズ batch size
args_bptt=35                 #通時バックプロパゲーション法のウインドウ幅 sequence length
args_dropout=0.2             #ドロップアウト率 dropout applied to layers (0 = no dropout)
args_tied=True               #この値が `真` であれば，入出力で同じ符号化を実行 tie the word embedding and softmax weights')
args_seed=1111               #乱数の種 random seed
args_cuda=False              #`真` なら GPU を使用する
args_log_interval=200        #中間の値を表示するタイミング report interval
args_save='model.pt'         #学習結果を保存するファイル名 path to save the final model
args_onnx_export=''          #onnx 出力するか否か `真` なら出力する path to export the final model in onnx format
args_nhead=2                 #多頭注意の 頭の数 the number of heads in the encoder/decoder of the transformer model
args_dry_run=False           #実行チェックのためのフラグ verify the code and the model
args_checkpoint='./model.pt' #チェックポイントファイル名 model checkpoint to use
args_outf='generated.txt'    #結果を出力するファイル名 output file for generated text
args_words=1000              #結果を出力する総単語数 number of words to generate
args_temperature=1.0         #ソフトマックス関数の温度 高いと不安定，低いと決定論的 temperature - higher will increase diversity

In [2]:
basyo = """草の戸も　住替る代ぞ　ひなの家
行く春や　鳥啼魚の　目は泪
あらたうと　青葉若葉の　日の光
剃捨て　黒髪山に　衣更　曽良
しばらくは　瀧に籠るや　夏の初
かさねとは　八重撫子の　名成るべし
夏山に　足駄をおがむ　かどでかな
木啄も　庵はやぶらず　夏木立
野を横に　馬ひきむけよ　ほととぎす
田一枚　植えて立ち去る　柳かな
卯の花を　かざしに関の　晴着かな
風流の　初やおくの　田植うた
栗といふ文字は西の木と書きて
世の人の　見付けぬ花や　軒の栗
早苗とる　手もとや昔(むかし)　しのぶ摺
笈も太刀も　五月にかざれ　帋幟
笠嶋は　いづこさ月の　ぬかり道
桜より　松は二木を　三月越し
あやめ草　足に結(むすば)ん　草鞋の緒
松島や　鶴に身をかれ　ほととぎす
夏草や　兵どもが　夢の跡
卯の花に　兼房みゆる　白毛かな
五月雨の　降のこしてや　光堂
蚤虱　馬の尿する　枕もと
涼しさを　我宿にして　ねまるなり
這出でよ　かひやが下の　ひきの声
まゆはきを　俤にして　紅粉の花
蚕飼する　人は古代の　すがたかな
閑さや　岩にしみ入る　蝉の声
五月雨を　あつめてはやし　最上川
ありがたや　雪をかほらす　南谷　
涼しさや　ほの三か月の　羽黒山　
雲の峯　幾つ崩れて　月の山　
語られぬ　湯殿にぬらす　袂かな　
湯殿山　銭ふむ道の　泪かな　曽良
あつみ山や　吹浦かけて　夕すずみ　
暑き日を　海にいれたり　最上川
象潟や　雨に西施が　ねぶの花　
汐越や　鶴はぎぬれて　海涼し
象潟や　料理何くふ　神祭　曽良
蜑の家や　戸板を敷て　夕涼　
波こえぬ　契りありてや　みさごの巣
荒海や　佐渡によこたふ　天河
一家に　遊女もねたり　萩と月
わせの香や　分入右は　有磯海
塚も動け　我泣声は　秋の風
秋涼し　手ごとにむけや　瓜茄子
あかあかと　日はつれなくも　秋の風
しほらしき　名や小松ふく　萩すすき
むざんやな　甲の下の　きりぎりす
石山の　石より白し　秋の風
山中や　菊はたおらぬ　湯の匂
行行て　たふれ伏と　も萩の原
今日よりや　書付消さん　笠の露
終宵　秋風聞くや　うらの山
庭掃て　出でばや寺に　散柳
物書て　扇引きさく　なごりかな
月清し　遊行のもてる　砂の上　
名月や　北国日和　さだめなき
寂しさや　須磨にかちたる　浜の秋　
波の間や　小貝にまじる　萩の塵
蛤の　ふたみにわかれ　行く秋ぞ"""

basho = basyo.split('\n')
#for ku in basho[:3]:
#    print([c for c in ku])
#    #print(ku)

fname = 'train.txt'
with open(fname, 'w') as f:
    for ku in basho[:50]:
        ku_ = [ch + ' ' for ch in ku]
        f.writelines(ku_); f.writelines('\n')

fname = 'valid.txt'
with open(fname, 'w') as f:
    for ku in basho[50:]:
        ku_ = [ch + ' ' for ch in ku]
        f.writelines(ku_); f.writelines('\n')

fname = 'test.txt'
with open(fname, 'w') as f:
    for ku in basho[50:]:
        ku_ = [ch + ' ' for ch in ku]
        f.writelines(ku_); f.writelines('\n')
        

In [3]:
!head test.txt

石 山 の 　 石 よ り 白 し 　 秋 の 風 
山 中 や 　 菊 は た お ら ぬ 　 湯 の 匂 
行 行 て 　 た ふ れ 伏 と 　 も 萩 の 原 
今 日 よ り や 　 書 付 消 さ ん 　 笠 の 露 
終 宵 　 秋 風 聞 く や 　 う ら の 山 
庭 掃 て 　 出 で ば や 寺 に 　 散 柳 
物 書 て 　 扇 引 き さ く 　 な ご り か な 
月 清 し 　 遊 行 の も て る 　 砂 の 上 　 
名 月 や 　 北 国 日 和 　 さ だ め な き 
寂 し さ や 　 須 磨 に か ち た る 　 浜 の 秋 　 


In [4]:
# 言語モデル PyTorch 版
# original: github pytorch examples, word language model/data.py
import os
from io import open
import torch

class Dictionary(object):
    """
    単語から ID を，ID から単語を得るためのクラス
    """
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)


class Corpus(object):
    """
    訓練に必要なコーパスを用意するためのクラス
    path で指定したディレクトリに train.txt, valid.txt, test.txt の 3 つファイルを用意
    しておく必要がある
    """
    
    def __init__(self, path):
        self.dictionary = Dictionary()
        self.train = self.tokenize(os.path.join(path, 'train.txt'))
        self.valid = self.tokenize(os.path.join(path, 'valid.txt'))
        self.test = self.tokenize(os.path.join(path, 'test.txt'))

    def tokenize(self, path):
        """Tokenizes a text file."""
        assert os.path.exists(path)
        # Add words to the dictionary
        with open(path, 'r', encoding="utf8") as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    self.dictionary.add_word(word)

        # Tokenize file content
        with open(path, 'r', encoding="utf8") as f:
            idss = []
            for line in f:
                words = line.split() + ['<eos>']
                ids = []
                for word in words:
                    ids.append(self.dictionary.word2idx[word])
                idss.append(torch.tensor(ids).type(torch.int64))
            ids = torch.cat(idss)

        return ids

In [5]:
# 言語モデル PyTorch 版
# original: github pytorch examples, word language model/model.py
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class RNNModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5, tie_weights=False):
        super(RNNModel, self).__init__()
        self.ntoken = ntoken
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
        else:
            try:
                nonlinearity = {'RNN_TANH': 'tanh', 'RNN_RELU': 'relu'}[rnn_type]
            except KeyError:
                raise ValueError( """An invalid option for `--model` was supplied,
                                 options are ['LSTM', 'GRU', 'RNN_TANH' or 'RNN_RELU']""")
            self.rnn = nn.RNN(ninp, nhid, nlayers, nonlinearity=nonlinearity, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        # Optionally tie weights as in:
        # "Using the Output Embedding to Improve Language Models" (Press & Wolf 2016)
        # https://arxiv.org/abs/1608.05859
        # and
        # "Tying Word Vectors and Word Classifiers: A Loss Framework for Language Modeling" (Inan et al. 2016)
        # https://arxiv.org/abs/1611.01462
        if tie_weights:
            if nhid != ninp:
                raise ValueError('When using the tied flag, nhid must be equal to emsize')
            self.decoder.weight = self.encoder.weight

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.encoder.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.weight)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output)
        decoded = decoded.view(-1, self.ntoken)
        return F.log_softmax(decoded, dim=1), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        if self.rnn_type == 'LSTM':
            return (weight.new_zeros(self.nlayers, bsz, self.nhid),
                    weight.new_zeros(self.nlayers, bsz, self.nhid))
        else:
            return weight.new_zeros(self.nlayers, bsz, self.nhid)


# Temporarily leave PositionalEncoding module here. Will be moved somewhere else.
class PositionalEncoding(nn.Module):
    r"""Inject some information about the relative or absolute position of the tokens
        in the sequence. The positional encodings have the same dimension as
        the embeddings, so that the two can be summed. Here, we use sine and cosine
        functions of different frequencies.
    .. math::
        \text{PosEncoder}(pos, 2i) = sin(pos/10000^(2i/d_model))
        \text{PosEncoder}(pos, 2i+1) = cos(pos/10000^(2i/d_model))
        \text{where pos is the word position and i is the embed idx)
    Args:
        d_model: the embed dim (required).
        dropout: the dropout value (default=0.1).
        max_len: the max. length of the incoming sequence (default=5000).
    Examples:
        >>> pos_encoder = PositionalEncoding(d_model)
    """

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        r"""Inputs of forward function
        Args:
            x: the sequence fed to the positional encoder model (required).
        Shape:
            x: [sequence length, batch size, embed dim]
            output: [sequence length, batch size, embed dim]
        Examples:
            >>> output = pos_encoder(x)
        """

        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


class TransformerModel(nn.Module):
    """Container module with an encoder, a recurrent or transformer module, and a decoder."""

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        try:
            from torch.nn import TransformerEncoder, TransformerEncoderLayer
        except:
            raise ImportError('TransformerEncoder module does not exist in PyTorch 1.1 or lower.')
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.encoder.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.weight)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, src, has_mask=True):
        if has_mask:
            device = src.device
            if self.src_mask is None or self.src_mask.size(0) != len(src):
                mask = self._generate_square_subsequent_mask(len(src)).to(device)
                self.src_mask = mask
        else:
            self.src_mask = None

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return F.log_softmax(output, dim=-1)


In [6]:
torch.manual_seed(args_seed)
if torch.cuda.is_available():
    if not args_cuda:
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")

device = torch.device("cuda" if args_cuda else "cpu")

In [7]:
###############################################################################
# Load data
###############################################################################
corpus = Corpus(args_data)

# Starting from sequential data, batchify arranges the dataset into columns.
# For instance, with the alphabet as the sequence and batch size 4, we'd get
# ┌ a g m s ┐
# │ b h n t │
# │ c i o u │
# │ d j p v │
# │ e k q w │
# └ f l r x ┘.
# These columns are treated as independent by the model, which means that the
# dependence of e. g. 'g' on 'f' can not be learned, but allows more efficient
# batch processing.

def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

eval_batch_size = 10
train_data = batchify(corpus.train, args_batch_size)
val_data = batchify(corpus.valid, eval_batch_size)
test_data = batchify(corpus.test, eval_batch_size)

In [8]:
###############################################################################
# Build the model
###############################################################################
ntokens = len(corpus.dictionary)
if args_model == 'Transformer':
    model = TransformerModel(ntokens, args_emsize, args_nhead, args_nhid, args_nlayers, args_dropout).to(device)
else:
    model = RNNModel(args_model, ntokens, args_emsize, args_nhid, args_nlayers, args_dropout, args_tied).to(device)

criterion = nn.NLLLoss()

In [9]:
###############################################################################
# Training code
###############################################################################

def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""

    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)


# get_batch subdivides the source data into chunks of length args.bptt.
# If source is equal to the example output of the batchify function, with
# a bptt-limit of 2, we'd get the following two Variables for i = 0:
# ┌ a g m s ┐ ┌ b h n t ┐
# └ b h n t ┘ └ c i o u ┘
# Note that despite the name of the function, the subdivison of data is not
# done along the batch dimension (i.e. dimension 1), since that was handled
# by the batchify function. The chunks are along dimension 0, corresponding
# to the seq_len dimension in the LSTM.
def get_batch(source, i):
    seq_len = min(args_bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target


def evaluate(data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0.
    ntokens = len(corpus.dictionary)
    if model != 'Transformer':
        hidden = model.init_hidden(eval_batch_size)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, args_bptt):
            data, targets = get_batch(data_source, i)
            if model == 'Transformer':
                output = model(data)
                output = output.view(-1, ntokens)
            else:
                output, hidden = model(data, hidden)
                hidden = repackage_hidden(hidden)
            total_loss += len(data) * criterion(output, targets).item()
    return total_loss / (len(data_source) - 1)


def train():
    # Turn on training mode which enables dropout.
    model.train()
    total_loss = 0.
    start_time = time.time()
    ntokens = len(corpus.dictionary)
    if model != 'Transformer':
        hidden = model.init_hidden(args_batch_size)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, args_bptt)):
        data, targets = get_batch(train_data, i)
        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        model.zero_grad()
        if model == 'Transformer':
            output = model(data)
            output = output.view(-1, ntokens)
        else:
            hidden = repackage_hidden(hidden)
            output, hidden = model(data, hidden)
        loss = criterion(output, targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), args_clip)
        for p in model.parameters():
            p.data.add_(p.grad, alpha=-lr)

        total_loss += loss.item()

        if batch % args_log_interval == 0 and batch > 0:
            cur_loss = total_loss / args_log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
                    'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // bptt, lr,
                elapsed * 1000 / log_interval, cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()
        if args_dry_run:
            break


def export_onnx(path, batch_size, seq_len):
    print('The model is also exported in ONNX format at {}'.
          format(os.path.realpath(args_onnx_export)))
    model.eval()
    dummy_input = torch.LongTensor(seq_len * args_batch_size).zero_().view(-1, args_batch_size).to(device)
    hidden = model.init_hidden(args_batch_size)
    torch.onnx.export(model, (dummy_input, hidden), path)


# Loop over epochs.
lr = args_lr
best_val_loss = None

In [ ]:
for epoch in range(1, args_epochs+1):
    #for epoch in range(1, 3):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    # Save the model if the validation loss is the best we've seen so far.
    if not best_val_loss or val_loss < best_val_loss:
        with open(args_save, 'wb') as f:
            torch.save(model, f)
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0


In [ ]:
# Load the best saved model.
with open(args_save, 'rb') as f:
    model = torch.load(f)
    # after load the rnn params are not a continuous chunk of memory
    # this makes them a continuous chunk, and will speed up forward pass
    # Currently, only rnn model supports flatten_parameters function.
    if model in ['RNN_TANH', 'RNN_RELU', 'LSTM', 'GRU']:
        model.rnn.flatten_parameters()

# Run on test data.
test_loss = evaluate(test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

if len(args_onnx_export) > 0:
    # Export the model in ONNX format.
    export_onnx(onnx_export, batch_size=1, seq_len=bptt)

| End of training | test loss  5.01 | test ppl   149.97


In [ ]:
# Set the random seed manually for reproducibility.
torch.manual_seed(args_seed)
if torch.cuda.is_available():
    if not args_cuda:
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")

device = torch.device("cuda" if args_cuda else "cpu")

with open(args_checkpoint, 'rb') as f:
    model = torch.load(f).to(device)
model.eval()

corpus = Corpus(args_data)
ntokens = len(corpus.dictionary)

is_transformer_model = hasattr(model, 'model_type') and model.model_type == 'Transformer'
if not is_transformer_model:
    hidden = model.init_hidden(1)
input = torch.randint(ntokens, (1, 1), dtype=torch.long).to(device)

with open(args_outf, 'w') as outf:
    with torch.no_grad():  # no tracking history
        for i in range(args_words):
            if is_transformer_model:
                output = model(input, False)
                word_weights = output[-1].squeeze().div(args.temperature).exp().cpu()
                word_idx = torch.multinomial(word_weights, 1)[0]
                word_tensor = torch.Tensor([[word_idx]]).long().to(device)
                input = torch.cat([input, word_tensor], 0)
            else:
                output, hidden = model(input, hidden)
                word_weights = output.squeeze().div(args_temperature).exp().cpu()
                word_idx = torch.multinomial(word_weights, 1)[0]
                input.fill_(word_idx)

            word = corpus.dictionary.idx2word[word_idx]
            outf.write(word + ('\n' if i % 20 == 19 else ' '))
            if i % args_log_interval == 0:
                print('| Generated {}/{} words'.format(i, args_words))                                                                                                

| Generated 0/1000 words
| Generated 200/1000 words
| Generated 400/1000 words
| Generated 600/1000 words
| Generated 800/1000 words


In [ ]:
from IPython.display import YouTubeVideo, display
tube_id = 'ek8zToZa7h0'  #https://www.youtube.com/watch?v=ek8zToZa7h0
#tube_id = 'inN8seMm7UI'  #https://www.youtube.com/watch?v=inN8seMm7UI
display(YouTubeVideo(tube_id, width=600, height=400))


In [ ]:
from IPython.display import IFrame
url = 'https://komazawa-deep-learning.github.io/'
url = 'https://insights.stackoverflow.com/trends?tags=python%2Cjavascript%2Cjava%2Cc%23%2Cphp%2Cc%2B%2B'
IFrame(url, width = 1276, height = 800)